In [57]:
from dotenv import load_dotenv
from ipywidgets import (
    interact, interactive, fixed, interact_manual, Layout
)
import ipywidgets as widgets
import json
import os
import requests

load_dotenv()

TOKEN = os.getenv('TOKEN')

HEADERS = {"Authorization": f"Bearer {TOKEN}"}

GRAPHQL_URL = "https://api.github.com/graphql"

In [67]:
start_date = widgets.DatePicker(
    description="Start Date",
    disabled=False
)
end_date = widgets.DatePicker(
    description="End Date",
    disabled=False
)

button = widgets.Button(
    description="Click me",
    disabled=False,
    button_style="", # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click me",
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

def fdate(start, end):
    print(
        "\nstart date: {}, end date: {}\n".format(start, end)
    )

date_out = widgets.interactive_output(
    fdate, {
        "start": start_date,
        "end": end_date
    }
)

display(start_date, end_date, button, date_out)

DatePicker(value=None, description='Start Date', step=1)

DatePicker(value=None, description='End Date', step=1)

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()

In [82]:
def get_unique_commit_authors_between_dates(since, until):
    """
    > Endpoint #1
    
    - Accepts: A start and end date.
    
    - Returns: A list of unique commit authors within the date constraints supplied.
    """
    q = '''{
      rateLimit {
        limit
        cost
        remaining
        resetAt
      }
      repository(name: "OpenRA", owner: "OpenRA") {
        ref(qualifiedName: "bleed") {
          target {
            ... on Commit {
              history(
                  first: 100
                  since: "2025-07-17T00:00:00"
                  until: "2025-07-18T00:00:00"
              ) {
                pageInfo {
                  hasNextPage
                  startCursor
                  endCursor
                }
                edges {
                  node {
                    author {
                      name
                      email
    }}}}}}}}}'''
    try:
        r = requests.post(
            GRAPHQL_URL,
            json={"query": q},
            headers=HEADERS
        )
        print(r.status_code)
        print(r.json())
    except Exception as e:
        print(e)

get_unique_commit_authors_between_dates(f"{start_date.value}T00:00:00", f"{end_date.value}T00:00:00")

200
{'data': {'rateLimit': {'limit': 5000, 'cost': 1, 'remaining': 4997, 'resetAt': '2025-07-18T18:46:20Z'}, 'repository': {'ref': {'target': {'history': {'pageInfo': {'hasNextPage': False, 'startCursor': 'a4966bc299919426fd77e5272bb7aac6f30ab73c 0', 'endCursor': 'a4966bc299919426fd77e5272bb7aac6f30ab73c 6'}, 'edges': [{'node': {'author': {'name': 'Vapre', 'email': 'anvilvapre@users.noreply.github.com'}}}, {'node': {'author': {'name': 'Matthias Mailänder', 'email': 'matthias@mailaender.name'}}}, {'node': {'author': {'name': 'Matthias Mailänder', 'email': 'matthias@mailaender.name'}}}, {'node': {'author': {'name': 'Ashley Newson', 'email': 'ashleynewson@smartsim.org.uk'}}}, {'node': {'author': {'name': 'Ashley Newson', 'email': 'ashleynewson@smartsim.org.uk'}}}, {'node': {'author': {'name': 'darkademic', 'email': '41052878+darkademic@users.noreply.github.com'}}}, {'node': {'author': {'name': 'darkademic', 'email': '41052878+darkademic@users.noreply.github.com'}}}]}}}}}}
